In [128]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset, DataLoader, TensorDataset)
import tqdm

from torchvision.datasets import FashionMNIST, ImageFolder
from torchvision import transforms, models

In [129]:
# Bring the Training-Data.
# Create DataSet that is the Initial-State as a PIL Image-Shape.
train_data = FashionMNIST(
    "./04/FashionMNIST",
    train=True, download=True,
    transform=transforms.ToTensor())

In [130]:
# Bring vertify-Data.
test_data = FashionMNIST(
    "../04/FashionMNIST",
    train=False, download=True,
    transform=transforms.ToTensor()
)

In [172]:
# Create each of DataLoader which Batch-Size are 128.
batch_size = 128
train_loader = DataLoader(fashion_mnist_train,
                         batch_size=batch_size, shuffle=True)
test_loader = DataLoader(fashion_mnist_test,
                        batch_size=batch_size, shuffle=False)


In [173]:
class FlatternLayer(nn.Module):
    def forward(self, x):
        sizes = x.size()
        return x.view(sizes[0], -1)

In [174]:
# Identify valid Layer.
class IdentifyLayer(nn.Module):
    def forward(self, x):
        return x

In [175]:
net = models.resnet18(pretrained=True)
for p in net.parameters():
    p.requires_grad=False
    print("Current situation in net - {0} and p - {1}".format(net, p))

Current situation in net - ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU

In [176]:
# net.fc calls IdentifyLayer Class.
net.fc = IdentifyLayer()

In [190]:
conv_net = nn.Sequential(
    nn.Conv2d(3, 32, 5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.Dropout2d(0.25),
    
    nn.Conv2d(32, 64, 5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.Dropout2d(0.25),
    
    nn.Conv2d(64, 128, 5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(128),
    nn.Dropout2d(0.25),
    
    FlatternLayer()
)

In [195]:
# Check the Image-Size by inputed Convolution-Product of Dummy-Data.
test_input = torch.ones(1, 3, 224, 224)
conv_output_size = conv_net(test_input).size()[-1]

In [183]:
# The Final CNN
net = nn.Sequential(
    conv_net,
    mlp
)
print(net)

# Evaluate-Function Helper
def eval_net(net, data_loader, device="cpu"):
    # Invalidation Dropout and BatchNorm
    net.eval()
    ys = []
    ypreds = []
    
    for x, y, in data_loader:
        # Send to Device that executes calculations to the 'to method'.
        x = x.to(device)
        y = y.to(device)
        
        # Because forward calculations are all here, 
        # the processing required for automatic differentials is set to off 
        # to limit unnecessary calculations.
        with torch.no_grad():
            _y, y_pred = net(x).max(3)
        ys.append(y)
        ypreds.append(y_pred)
        
    # The results of the prediction of the mini-deployment 
    # unit are grouped together.
    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)
    
    # Calculate Prediction-Accuracy
    acc = (ys == ypreds).float().sum() / len(ys)
    return acc.item()

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
    (3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Dropout2d(p=0.25)
    (5): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): ReLU()
    (8): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Dropout2d(p=0.25)
    (10): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): ReLU()
    (13): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): Dropout2d(p=0.25)
    (15): FlatternLayer()
  )
  (1): Sequential(
    (0): Linear(in_features=73728, out_features=200, bias=True)
    (1): ReLU()
    (2): 

In [193]:
# Training-Function Helper
def train_net(net, train_loader, test_loader,
             optimizer_cls=optim.Adam,
             loss_fn=nn.CrossEntropyLoss(),
             only_fc=False,
             n_iter=10, device="cpu"):
    train_losses = []
    train_acc = []
    val_acc = []
    optimizer = optimizer_cls(net.parameters())
    
    for epoch in range(n_iter):
        running_loss = 0.0
        
        # Set Neura-Network as Training-Mode.
        net.train()
        n = 1
        n_acc = 0
    
        
        # It takes a lot of time, 
        # so use tqdm to display Progress-Bar.
        for i, (xx, yy) in tqdm.tqdm(
            enumerate(train_loader),
            total=len(train_loader)):
            xx = xx.to(device)
            yy = yy.to(device)
            
            h = net(xx)
            print("xx shape : ", xx.size())
            loss = loss_fn(h, yy)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            n += len(xx)
            _, y_pred = h.max(1)
            n_acc += (yy == y_pred).float().sum().item()
        
        train_losses.append(running_loss / i)
        # Prediction Accuracy of Train-Data.
        train_acc.append(n_acc / n)
        
        # Prediction Accuracy of Verify-Data.
        val_acc.append(eval_net(net, test_loader, device))
        
        # Display epoch result.
        print(epoch, train_losses[-1], train_acc[-1],
             val_acc[-1], flush=True)

# Send to all of Neural-Network Parameter to the GPU.
# net.to("cuda:0")

# Send to all of Neural-Network Parameter to the CPU.
net.to("cpu")

# Training Execution
# train_net(net, train_loader, test_loader, n_iter=10, only_fc=False,
#          device="cpu")

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
    (3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Dropout2d(p=0.25)
    (5): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): ReLU()
    (8): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Dropout2d(p=0.25)
    (10): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): ReLU()
    (13): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): Dropout2d(p=0.25)
    (15): FlatternLayer()
  )
  (1): Sequential(
    (0): Linear(in_features=73728, out_features=200, bias=True)
    (1): ReLU()
    (2): 